In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdate
import matplotlib.ticker as mtick
import os
from openpyxl import load_workbook
%matplotlib inline

In [55]:
def new_data(path,unit = 1, index=None, groupby=None):
    '''导入数据，并分别生成gmv,sales和atv三个数据表
    '''
    data = pd.read_excel(path, index_col=index)
    data['gmv'] = data['gmv'] / unit
    data['sale_qtty'] = data['sale_qtty'] /unit
    timedata = data.groupby(groupby)
    new_data = timedata.sum()
    return new_data
test = new_data('./JD/JD_cats.xlsx',groupby = ['dt','cid1_name','source'])
test.loc[(slice(None),slice(None),'pop'),'gmv'].unstack()

source                         pop
dt         cid1_name              
2018-01-01 其他         1.623933e+08
           医药保健       7.771367e+08
           宠物生活       1.018186e+08
           家居家装       3.181759e+09
           家用电器       2.708908e+09
           影视音乐       6.771447e+06
           手机数码       1.786875e+09
           服装配饰       3.197353e+09
           母婴用品       8.682984e+08
           汽车相关       5.394938e+08
           玩具乐器       3.241043e+08
           珠宝首饰       6.212276e+08
           电脑、办公      2.420675e+09
           箱包配饰       5.996350e+06
           美妆个护       1.098426e+09
           运动户外       1.233307e+09
           酒类         7.295925e+08
           鞋类         8.427353e+08
           食品饮料及生鲜    1.555892e+09
2018-02-01 其他         9.913150e+07
           医药保健       4.804576e+08
           宠物生活       6.697347e+07
           家居家装       1.426577e+09
           家用电器       1.590460e+09
           影视音乐       4.000390e+06
           手机数码       1.300235e+09
           服装配饰       1.168477e+09
           母婴用品       5.514015e+08
           汽车相关       2.825442e+08
           玩具乐器       2.392688e+08
...                            ...
2019-04-01 母婴用品       9.580590e+08
           汽车相关       5.353543e+08
           玩具乐器       2.990317e+08
           珠宝首饰       6.296159e+08
           电脑、办公      2.265517e+09
           箱包配饰       2.541861e+08
           美妆个护       8.794447e+08
           运动户外       1.743084e+09
           酒类         3.466333e+08
           鞋类         6.300565e+08
           食品饮料及生鲜    1.383103e+09
2019-05-01 其他         2.759753e+08
           医药保健       1.373608e+09
           宠物生活       1.601435e+08
           家居家装       3.803358e+09
           家用电器       3.346638e+09
           影视音乐       5.151500e+04
           手机数码       2.117006e+09
           服装配饰       2.033305e+09
           母婴用品       9.829268e+08
           汽车相关       5.542579e+08
           玩具乐器       3.897199e+08
           珠宝首饰       7.871221e+08
           电脑、办公      2.240956e+09
           箱包配饰       2.644693e+08
           美妆个护       1.021481e+09
           运动户外       1.680144e+09
           酒类         3.608202e+08
           鞋类         6.035692e+08
           食品饮料及生鲜    1.704086e+09

[323 rows x 1 columns]

In [58]:
a = ['a','b']
a.insert(1,'c')
a

['a', 'c', 'b']

* load data, and seperate GMV, Sales, and ATV

In [2]:
def load_data(path,unit = 1, index=None, growpby=None):
    '''导入数据，并分别生成gmv,sales和atv三个数据表
    '''
    data = pd.read_excel(path, index_col=index)
    data['gmv'] = data['gmv'] / unit
    data['sale_qtty'] = data['sale_qtty'] /unit
    timedata = data.groupby(growpby)
    new_data = timedata.sum()
    gmv = new_data['gmv'].unstack()
    sales = new_data['sale_qtty'].unstack()
    atv = gmv / sales
    return gmv, sales, atv

* load brands or catelog list

In [3]:
def load_cat(path,name):
    '''导入品牌目录，并根据品类（name）获取品牌列表
    '''
    brands = pd.read_excel(path).fillna(0)
    brand_lit =brands[name].tolist()
    while 0 in brand_lit:
        brand_lit.remove(0)
    return brand_lit

* calculate the growth

In [4]:
def yoy_growth(df,period):
    '''计算yoy增长率
    '''
    tmp = df.to_period(period).groupby('dt').sum()
    tmp1 = tmp['2018']
    tmp2 =tmp['2019']
    growth = tmp2.copy()
    m,n = tmp2.shape
    for i in range(n):
        for j in range(m):
            growth.iloc[j,i] =(tmp2.iloc[j,i]/tmp1.iloc[j,i]-1)
    return growth

In [5]:
def cr_growth(df,period):
    '''计算环比增长率
    '''
    tmp = df.to_period(period).groupby('dt').sum()
    growth = tmp.copy()
    m,n = tmp.shape
    for i in range(n):
            growth.iloc[:,i] = (tmp.iloc[:,i]/tmp.iloc[:,i].shift(1) -1)
    return growth

* save datas

In [6]:
def bubble_data(gmv,date,kind = None,cat_gmv = None,brands = None):
    '''格式化需要画泡泡图的数据
    input:  gmv（df）：需要画泡泡图的数据
            date(str): 日期 ‘2019-04-01’
            kind(str): 数据的类型（cat = '分品类'，其他表示具体的品牌数据）
            cat_gmv(df): 品类的总数据；当cat为品牌时，必选。
            brands(str)：品类的名称，当cat为品牌时，必选。
    output: bubble(df): 返回泡泡数据，份额、增速和gmv
    '''
    if kind == "大家电" or kind == "小家电":
        kind = '家用电器'
    if kind == "cat":
        data = gmv.loc[date,:]
        total = data.sum()
        gmv_pct = data / total
        gmv_yoy = yoy_growth(gmv,"M").loc[date,:]
    else:
        total = cat_gmv.loc[date,kind]
        data = gmv[brands].loc[date,:]
        gmv_pct = data / total
        gmv_yoy = yoy_growth(gmv,"M")[brands].loc[date,:]
    bubble = pd.DataFrame(gmv_pct)
    bubble.columns = ['平均市场份额']
    bubble["销售额增速"] = np.array(gmv_yoy)
    bubble["销售额"] = np.array(data)
    return bubble

def save_bubble(gmv,cat_gmv,catalog,catlist,date,bubblepath):
    '''保存bubble数据
    '''
    writer = pd.ExcelWriter(date + bubblepath)
    cat_bubble = bubble_data(cat_gmv,date,kind='cat')
    cat_bubble.to_excel(writer,sheet_name = '平台')
    for kind in catlist:
        brands = load_cat(catalog,kind)
        b_brand = bubble_data(gmv,date,kind = kind,cat_gmv = cat_gmv,brands = brands)
        b_brand.to_excel(writer,sheet_name=kind)
    writer.save()
    writer.close()

In [7]:
def save_cat_sheet(df,path,name):
    '''保存品类数据
    input: df（df）: 需要整理的原始数据
            path(str)：保存的路径
            name: sheet 名
    '''
    n = df.shape[0]
    my_growth = yoy_growth(df,'M')
    qy_growth = yoy_growth(df,"Q")
    c_growth = cr_growth(df,"M")

    workbook = load_workbook(path)
    writer = pd.ExcelWriter(path,engine='openpyxl')
    writer.book = workbook
    if name in workbook.sheetnames:
        workbook.remove(workbook[name])
        workbook.save(path)
    df.to_excel(writer,sheet_name=name)
    my_growth.to_excel(writer,sheet_name=name,startrow=n+3)
    qy_growth.to_excel(writer,sheet_name=name,startrow=2*n+3)
    c_growth.to_excel(writer,sheet_name=name,startrow=3*n+3)
    writer.save()
    writer.close()
    
def save_cat(path):
    save_cat_sheet(cat_gmv,path,'gmv')
    save_cat_sheet(cat_sales,path,'sales')
    save_cat_sheet(cat_atv,path,'atv')

In [136]:
def save_brand(cat,path,cat_path,gmv, sales, atv,gmv_g,sales_g,atv_g):
    '''保存品牌数据
    input: cat（str）: 需要整理的品类的名称
            path(str)：保存的路径
    '''
    brands = load_cat(cat_path,cat)  
    gmv = gmv[brands]
    sales = sales[brands]
    atv = atv[brands]
    gmv_g = gmv_g[brands]
    sales_g = sales_g[brands]
    atv_g = atv_g[brands]
    
    m,n = gmv.shape
    workbook = load_workbook(path)
    writer = pd.ExcelWriter(path,engine='openpyxl')
    writer.book = workbook
    if cat in workbook.sheetnames:
        workbook.remove(workbook[cat])
        workbook.save(path)
    
    gmv.to_excel(writer,sheet_name=cat)
    gmv_g.to_excel(writer,sheet_name=cat,startcol = n+3 )
    
    sales.to_excel(writer,sheet_name=cat,startrow=m+3)
    sales_g.to_excel(writer,sheet_name=cat,startrow=m+3,startcol=n+3)
    
    atv.to_excel(writer,sheet_name=cat,startrow=2*m+6)
    atv_g.to_excel(writer,sheet_name=cat,startrow=2*m+6,startcol=n+3)
    writer.save()
    writer.close()

* 运行

In [39]:
unit = 100000000
cat_gmv, cat_sales, cat_atv = load_data('Tmall_cat.xlsx', index='dt', growpby=['dt','cid1_name'],unit = unit)
gmv, sales, atv = load_data('Tmall_brand.xlsx', index='dt', growpby=['dt', 'main_brand_name'],unit=unit)

In [40]:
gmv_g = yoy_growth(gmv,"M")
sales_g = yoy_growth(sales,"M")
atv_g = yoy_growth(atv,"M")

D:\anaconda\envs\dataServer\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [32]:
catlist = ['医药保健','酒类','大家电','小家电','美妆个护','服装鞋包']

In [36]:
path = 'Tmall_cat.xlsx'
save_cat(path)

In [45]:
brand_path = "Tmall_brand.xlsx"
cat_path = 'brand_catalog.xlsx'
for cat in catlist:
    save_brand(cat,brand_path,gmv, sales, atv,gmv_g,sales_g,atv_g)

In [150]:
date = '2019-04-01'
bubblepath = 'bubble_data.xlsx'
catalog = 'brand_catalog.xlsx'
save_bubble(gmv,cat_gmv,catalog,catlist,date,bubblepath)

D:\anaconda\envs\dataServer\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [9]:
unit = 100000000
date = '2019-04-01'
# 1.导入品类和品牌的原始数据
brand_path_D = "./JD/JD_brands.xlsx"
cat_path_D = './JD/JD_cats.xlsx'
cat_gmv_D, cat_sales_D, cat_atv_D = load_data(
    cat_path_D, index='dt', growpby=[
        'dt', 'cid1_name','source'], unit=unit)
gmv_D, sales_D, atv_D = load_data(
    brand_path_D, index='dt', growpby=[
        'dt', 'main_brand_name','source'], unit=unit)

In [13]:
# 2. 计算增长率
gmv_g_D = yoy_growth(gmv_D, "M")
sales_g_D = yoy_growth(sales_D, "M")
atv_g_D = yoy_growth(atv_D, "M")

AttributeError: 'MultiIndex' object has no attribute 'to_period'

In [ ]:
    # 2. 计算增长率
    gmv_g_D = yoy_growth(gmv_D, "M")
    sales_g_D = yoy_growth(sales_D, "M")
    atv_g_D = yoy_growth(atv_D, "M")

    # 3. 保存品类数据

    save_cat_sheet(cat_gmv_D, cat_path_D, 'gmv')
    save_cat_sheet(cat_sales_D, cat_path_D, 'sales')
    save_cat_sheet(cat_atv_D, cat_path_D, 'atv')

    # 4. 保存品牌数据
    catlist_D = ['医药保健', '酒类', '大家电', '小家电', '食品饮料及生鲜']
    cat_name_path_D = './JD/JD_catalog.xlsx'
    for cat in catlist_D:
        save_brand(
            cat,
            brand_path_D,
            cat_name_path_D,
            gmv_D,
            sales_D,
            atv_D,
            gmv_g_D,
            sales_g_D,
            atv_g_D)

    # 5. 保存泡泡图格式数据
    bubblepath_D = 'bubble_data_JD.xlsx'
    save_bubble(
        gmv_D,
        cat_gmv_D,
        cat_name_path_D,
        catlist_D,
        date,
        bubblepath_D,
        "./JD/")